In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
662568,2022-02-28,0-9
662569,2022-02-28,30-39
662570,2022-02-28,40-49
662571,2022-02-28,10-19_
662572,2022-02-28,10-19_


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-02-24,341,462,362,489,372,192,157,72,30,9
2022-02-25,357,384,280,390,342,194,117,100,34,12
2022-02-26,294,347,244,317,246,173,100,56,21,12
2022-02-27,252,388,200,246,237,131,91,61,38,5
2022-02-28,212,360,202,219,205,125,87,49,23,14


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

1496

In [7]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [8]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
81,2022-02-24,348,444
82,2022-02-25,341,462
83,2022-02-26,357,384
84,2022-02-27,294,347
85,2022-02-28,252,388


In [9]:
df_0_19.to_csv("../data/cases_0_19.csv")